In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import numpy as np
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains, Keys
from webdriver_manager.chrome import ChromeDriverManager

#from webdriver_manager.chrome import ChromeDriverManager # Chrome driver 자동 업데이트


import pyperclip


In [2]:
ID = '#############'
PW = '##################'

In [3]:


chrome_options = Options()
#chrome_options.add_argument("--disable-javascript")

driver = webdriver.Chrome()



In [4]:
# 크롬드라이버 실행
#driver = webdriver.Chrome() 

#크롬 드라이버에 url 주소 넣고 실행
driver.get('https://serieson.naver.com/v3/movie/products/drama?sortType=UPDATE_DESC&price=all')

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

In [14]:
import re

def make_movie_list():
    return driver.find_elements(By.CLASS_NAME, "ListCollection_movie_item__eUfF0")
    
    

def click_movie(li_element, index):
    li_element = driver.find_elements(By.CLASS_NAME, "ListCollection_movie_item__eUfF0")
        
    for element in range(index, len(li_element)):
        try :     
            a_element = li_element[element].find_element(By.TAG_NAME, "a").click()

            time.sleep(1)

            crawl_data()
            driver.back()
            time.sleep(2)
        
        except Exception as e :
            print('에러 발생, 에러 내용 ')
            print(e)
            driver.back()
            return element

    return -1


def crawl_data():
    
    title = driver.find_element(By.XPATH,'/html/body/div/div[2]/div[1]/div[2]/div/div[1]/div[1]/strong').text
    lists = driver.find_elements(By.XPATH,"/html/body/div/div[2]/div[1]/div[2]/div/div[1]/div[2]//span")
    director_list = driver.find_elements(By.XPATH,'/html/body/div/div[2]/div[1]/div[2]/ul/li[1]/div[1]/div//span')
    actors_list = driver.find_elements(By.XPATH,'/html/body/div/div[2]/div[1]/div[2]/ul/li[1]/div[2]/div//span')
    subtitle = driver.find_element(By.XPATH,'/html/body/div/div[2]/div[1]/div[2]/div/div[1]/div[1]/span[1]/span').text
    dubbing = driver.find_element(By.XPATH,'/html/body/div/div[2]/div[1]/div[2]/div/div[1]/div[1]/span[2]').text
    like = driver.find_element(By.XPATH,'/html/body/div/div[2]/div[1]/div[2]/div/ul/li[2]/div/a/em[2]').text
    comment = driver.find_element(By.XPATH,'/html/body/div/div[2]/div[1]/div[2]/div/ul/li[3]/button').text

    try:
        period = driver.find_element(By.XPATH,'/html/body/div/div[2]/div[1]/div[2]/ul/li[3]/div/div[1]/div/span[2]').text
    except:
        period = driver.find_element(By.XPATH,'/html/body/div/div[2]/div[1]/div[2]/ul/li[2]/div/div[1]/div/span[2]').text

    age = None
    runtime = None
    isDRM = None
    rating = None
    year = None
    director = ''
    actors = ''
    tmp = True

    try :
        type_cost = driver.find_element(By.XPATH,'/html/body/div/div[2]/div[1]/div[2]/div/div[2]/button').text
    except Exception as e:
        type_cost = None
        tmp=False

    if subtitle:
        version = subtitle
    elif dubbing:
        version = dubbing
    else:
        version = None

    if like:
        like = like
    else:
        like = '0'

    if comment == '감상평':
        comment = '0'

    if period:
        period = period[5:]
    else:
        period = None

    try:
        for i in lists:
            target = i.text

            if target.endswith("관람가") or target.endswith('관람불가'):
                age = target
            elif target.endswith("분"):
                runtime = target
            elif target=='DRM':
                isDRM = target
            elif target =='스트리밍 전용':
                pass
            elif float(target)<11:
                rating = target
            else : 
                year = target
        if director_list:
            for dir in director_list:
                dir_name = dir.text
                director += dir_name
        else:
            director = None

        if actors_list:
            for act in actors_list:
                act_name = act.text
                actors += act_name
        else:
            actors = None

        arr_title.append(title)
        arr_age.append(age)
        arr_runtime.append(runtime)
        arr_isDRM.append(isDRM)
        arr_rating.append(rating)
        arr_year.append(year)
        arr_director.append(director)
        arr_actors.append(actors)
        if tmp:
            arr_type.append(type_cost[3:5])
            arr_cost.append(type_cost[6:])
        else :
            arr_type.append(None)
            arr_cost.append(None)
        
        arr_version.append(version)
        arr_like.append(like)
        arr_comment.append(comment)
        arr_period.append(period)
        #print(type_cost)
        #print(title, runtime, age, isDRM, rating, year, director, actors, type_cost[3:5], 
        #      type_cost[6:], version, like, comment, period)
    except:
        print("크롤링 작업 중 오류가 발생했습니다.")
        print(title, runtime, age, isDRM, rating, year, director, actors, type_cost[3:5], 
              type_cost[6:], version, like, comment, period)
    
    
    
    
    
def click_DBG():
    
    while True:
        
        try:
            driver.find_element(By.CLASS_NAME, "ViewMoreButton_view_more_button__eGoJu").click()
            time.sleep(1)

        except:
            print("더보기 버튼이 더 없습니다")
            break
    

def do_login():
    login_btn = driver.find_element(By.CSS_SELECTOR, '#gnb_login_button')
    login_btn.click()

    time.sleep(3)

    id = driver.find_element(By.CSS_SELECTOR, "#id")
    id.click()
    pyperclip.copy(ID)
    actions = ActionChains(driver)
    actions.key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(2)

    pw = driver.find_element(By.CSS_SELECTOR, "#pw")
    pw.click()
    pyperclip.copy(PW)
    actions = ActionChains(driver)
    actions.key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)


    btn_login = driver.find_element(By.CSS_SELECTOR, "#log\.login")
    btn_login.click()
    time.sleep(1)

In [13]:
last_index = 12615


In [6]:
last_index = 11571


arr_title = []
arr_age = []
arr_runtime = []
arr_isDRM = []
arr_rating = []
arr_year = []
arr_director = []
arr_actors = []
arr_type = []
arr_cost = []
arr_version = []
arr_like = []
arr_comment = []
arr_period = []

In [7]:
do_login()

In [68]:
#crawl_data()


# 로그인 후, 더보기가 없을 때까지 계속 누르기.
# 작업이 끝나면 전체 영화 리스트를 list로 저장
# 마지막으로 크롤링했던 영화 인덱스부터 movie_list를 순회하며 크롤링 진행

last index가 -1로 되면 성공적으로 종료된것, -1이 아니라면 그때부터 다시 시작하면 됨

In [15]:
%%time

while last_index != -1:
    click_DBG()
    movie_list = make_movie_list()

    last_index = click_movie(movie_list , last_index)

    if last_index == -1:
        print('모든 크롤링 작업이 완료되었습니다')
        break
    else :
        print(f'중간에 크롤링이 중단되었습니다. \n 중단된 영화 인덱스 : {last_index}')
        last_index+=1


더보기 버튼이 더 없습니다
에러 발생, 에러 내용 
Message: stale element reference: stale element not found
  (Session info: chrome=126.0.6478.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF704D63E52+31618]
	(No symbol) [0x00007FF704CDB0B9]
	(No symbol) [0x00007FF704B9888A]
	(No symbol) [0x00007FF704BAAF2D]
	(No symbol) [0x00007FF704BAB9BA]
	(No symbol) [0x00007FF704B9F71B]
	(No symbol) [0x00007FF704B9D486]
	(No symbol) [0x00007FF704BA1011]
	(No symbol) [0x00007FF704BA10B0]
	(No symbol) [0x00007FF704BE805C]
	(No symbol) [0x00007FF704BE862C]
	(No symbol) [0x00007FF704BDBCDC]
	(No symbol) [0x00007FF704C0D14F]
	(No symbol) [0x00007FF704BDBBA6]
	(No symbol) [0x00007FF704C0D320]
	(No symbol) [0x00007FF704C2CA80]
	(No symbol) [0x00007FF704C0CEB3]
	(No symbol) [0x00007FF704BDA46B]
	(No symbol) [0x00007FF704BDB001]
	GetHandleVerifier [0x00007FF70506A02D+3202397]
	G

In [32]:
# last_index = click_movie(movie_list , last_index)

# if last_index == -1:
#     print('모든 크롤링 작업이 완료되었습니다')
# else :
#     print(f'중간에 크롤링이 중단되었습니다. \n 중단된 영화 인덱스 : {last_index}')
    

In [16]:
df = pd.DataFrame({
    'title': arr_title,
    'age': arr_age,
    'runtime': arr_runtime,
    'rating': arr_rating,
    'like': arr_like,
    'comment': arr_comment,
    'year': arr_year,
    'isDRM': arr_isDRM,
    'director': arr_director,
    'actor': arr_actors,
    'version': arr_version,
    'type': arr_type,
    'cost': arr_cost,
    'period': arr_period
})

In [17]:
df

,title,age,runtime,rating,like,comment,year,isDRM,director,actor,version,type,cost,period
0,어떤 만남,청소년 관람불가,81분,7.66,35,7,2014,DRM,리자 아주엘로스,"소피 마르소,프랑수아 클루제,리자 아주엘로스,알렉상드르 아스티에,아서 벤자쿠엔,조나...",None,구매,"6,500",제한 없음
1,앨빈과 슈퍼밴드 2,전체 관람가,88분,8.88,13,1,2009,DRM,베티 토마스,"제이슨 리,저스틴 롱,매튜 그레이 구블러,제시 맥카트니,제커리 레비,데이빗 크로스,...",자막,구매,"5,500",5년
2,엑스 파일: 나는 믿고 싶다,15세 관람가,104분,5.51,3,0,2008,DRM,크리스 카터,"데이비드 듀코브니,질리언 앤더슨,아만다 피트,빌리 코놀리,엑지빗,미치 필레기",None,구매,"5,500",5년
3,어느 멋진 순간,15세 관람가,117분,7.82,49,3,2006,DRM,리들리 스콧,"러셀 크로우,마리옹 꼬띠아르,앨버트 피니,애비 코니쉬,톰 홀랜더,디디에 부르돈,프레...",None,구매,"5,500",5년
4,미스 리틀 선샤인,15세 관람가,102분,8.97,208,6,2006,DRM,"조나단 데이턴,발레리 페리스","스티브 카렐,토니 콜렛,그렉 키니어,폴 다노,아비게일 브레스린,알란 아킨,폴라 뉴좀...",None,구매,"5,500",5년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,미션 투 베를린,15세 관람가,123분,9.4,2,1,None,DRM,드로 자하비,"헤이노 페르치,벳티나 짐머만,얼리치 노던",None,구매,"1,500",제한 없음
1075,허당 해적단,전체 관람가,88분,7.97,9,0,None,DRM,"피터 로드,제프 뉴이트","휴 그랜트,브렌단 글리슨,제레미 피번,브라이언 블레시드,레니 헨리,셀마 헤이엑,마틴...",자막,구매,"5,000",5년
1076,에드 우드,15세 관람가,126분,8.65,24,2,None,DRM,팀 버튼,"조니 뎁,마틴 랜도,사라 제시카 파커,패트리샤 아퀘트,제프리 존스,G.D. 스프라들...",None,구매,"5,500",5년
1077,제임스와 거대한 복숭아(자막판),전체 관람가,76분,8.62,64,3,None,DRM,헨리 셀릭,"조안나 럼리,미리암 마고리스,피트 포스틀스웨이트,스티븐 컬프,토니 하니,알 날반디언...",None,구매,"5,500",5년


In [18]:
df.to_csv('drama_adf2.csv', index=False, encoding="utf-8-sig")

In [12]:
last_index

-1